In [ ]:
from google.colab import drive
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import spacy
from collections import Counter
import torch
import torchtext
import torch.nn as nn
import torch.optim as optim
import random

In [ ]:
!pip install torchtext==0.6.0 --quiet
from torchtext.data import Field, BucketIterator

In [ ]:
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# Read JSON data from file
with open('/gdrive/MyDrive/train_data1.json', 'r') as file:
    data = json.load(file)

In [ ]:
# Read JSON data from file
with open('/gdrive/MyDrive/test_data1_final.json', 'r') as file:
    test_data = json.load(file)

In [ ]:
import string

# Define the remove_punctuations function
def remove_punctuations(sentence):
    punctuations = list(string.punctuation)
    punctuations.append('।')
    punctuations.append('৷')
    punctuations.append('’')
    punctuations.append('‘')
    cleaned = ""
    for letter in sentence:
        if letter not in punctuations:
            cleaned += letter
    return cleaned


In [ ]:
import re

# Define a function to check if a sentence contains English words
def contains_english_words(sentence):
    return bool(re.search(r'[a-zA-Z]', sentence))

In [ ]:
# Process JSON data
source_sentences_train = []
target_sentences_train = []
len_gujrati = []
len_english = []

id_train = []

for language_pair, language_data in data.items():
    if(language_pair == "English-Gujarati"):
      print(f"Language Pair: {language_pair}")
      for data_type, data_entries in language_data.items():
          print(f"  Data Type: {data_type}")
          for entry_id, entry_data in data_entries.items():
              source = entry_data["source"].lower()
              target = entry_data["target"].lower()
              source = remove_punctuations(source)
              target = remove_punctuations(target)
              if not contains_english_words(target):
                source_sentences_train.append(source)
                target_sentences_train.append(target)
                id_train.append(entry_id)
                len_gujrati.append(len(source.split(' ')))
                len_english.append(len(target.split(' ')))

Language Pair: English-Gujarati
  Data Type: Train


In [ ]:
# Calculate the maximum length of gujrati and English sentences
max_length_gujrati = max(len_gujrati)
max_length_english = max(len_english)

print(f"Maximum Length of gujrati Sentence: {max_length_gujrati} words")
print(f"Maximum Length of English Sentence: {max_length_english} words")

Maximum Length of gujrati Sentence: 96 words
Maximum Length of English Sentence: 93 words


In [ ]:
len(target_sentences_train)

46848

In [ ]:
test_source = []
valid_id = []
len_test=[]


for language_pair, language_data in test_data.items():
    if(language_pair == "English-Gujarati"):
      print(f"Language Pair: {language_pair}")
      for data_type, data_entries in language_data.items():
          print(f"  Data Type: {data_type}")
          for entry_id, entry_data in data_entries.items():
              source = entry_data["source"].lower()
              source = remove_punctuations(source)
              test_source.append(source)
              valid_id.append(entry_id)
              len_test.append(len(source.split(' ')))

Language Pair: English-Gujarati
  Data Type: Test


In [ ]:
len(test_source)

13567

In [ ]:
print(max(len_test))

71


In [ ]:
nlp_en = spacy.load('en_core_web_sm')
# Tokenize English sentences
tokenized_source_sentences = []
for sentence in source_sentences_train:
    doc = nlp_en(sentence)
    tokens = [token.text for token in doc]
    tokenized_source_sentences.append(tokens)

In [ ]:
tokenized_test_source = []
for sentence in test_source:
    doc = nlp_en(sentence)
    tokens = [token.text for token in doc]
    tokenized_test_source.append(tokens)

In [ ]:
!pip install nltk

In [ ]:
pip install indic-nlp-library

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 14.7 MB/s eta 0:00:00


In [ ]:
import nltk

nltk.download("indic_nltk")

from indicnlp.tokenize import indic_tokenize


# Tokenize gujrati sentences
tokenized_target_sentences = []
for sentence in target_sentences_train:
    tokens = indic_tokenize.trivial_tokenize(sentence)
    tokenized_target_sentences.append(tokens)

[nltk_data] Error loading indic_nltk: Package 'indic_nltk' not found
[nltk_data]     in index


In [ ]:
from torchtext.data import Dataset, Example, Field

def tokenize_english(text):
    return text

def tokenize_gujrati(text):
    return text

class CustomTranslationDataset(Dataset):
    def __init__(self, source_sentences, target_sentences, source_field, target_field):
        fields = [('source', source_field), ('target', target_field)]
        examples = []
        for src, tgt in zip(source_sentences, target_sentences):
            examples.append(Example.fromlist([src, tgt], fields))
        super().__init__(examples, fields)

# Create Fields for English (source) and gujrati (target) text
english = Field(tokenize=tokenize_english,
                lower=True,
                init_token="<sos>",
                eos_token="<eos>")

gujrati = Field(tokenize=tokenize_gujrati,
              lower=True,
              init_token="<sos>",
              eos_token="<eos>")


train_dataset = CustomTranslationDataset(tokenized_source_sentences, tokenized_target_sentences, english, gujrati)

english.build_vocab(train_dataset,max_size=30000, min_freq=3)
gujrati.build_vocab(train_dataset,max_size=30000, min_freq=3)



In [ ]:
print(len(english.vocab), len(gujrati.vocab))

16201 22397


In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 32

train_iterator = BucketIterator(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    sort_within_batch=True,
    sort_key=lambda x: len(x.source),
    device=device
)


In [ ]:
count = 0
for data in train_iterator:
  if count < 1 :
    print("Shapes", data.source.shape, data.target.shape)
    print()
    print("English - ",*data.source, " Length - ", len(data.source))
    print()
    print("gujrati - ",*data.target, " Length - ", len(data.target))
    temp_eng = data.source
    temp_gujrati = data.target
    count += 1

Shapes torch.Size([21, 32]) torch.Size([25, 32])

English -  tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([    0,     6,    16,    26, 11358,    46,    28, 11939,   302,   336,
           38,     0,     4,    74,    28,   214,     4,   569,     6,   375,
          802,   212,  9532,     6,   228, 14068,  3346,  3469,  1079,   533,
            4,     6], device='cuda:0') tensor([  901,  9765,   200,    14,     0,    81,    60,    35,   290,   815,
         1104,  1187,   865,    29,    13,    58,   154,     4,     4,     8,
           77,     6,     8,     4,   478,  6667,     4,    29,    67,     4,
        16200,   491], device='cuda:0') tensor([ 2241,     0,    29,    56,     9,   915, 10469,    41,  8399,     5,
         1369,     0,   316,  1525,   152,   366,     5,   673, 10888,     4,
          386,    34,    11,  1347,    23, 12640,  5579,    58,    20,   253,
         1982,    10], devi

In [ ]:
temp_eng_idx = (temp_eng).cpu().detach().numpy()
temp_gujrati_idx = (temp_gujrati).cpu().detach().numpy()

In [ ]:
a = list(english.vocab.__dict__.values())
word_2_idx = dict(a[3])
idx_2_word = {}
for i,j in word_2_idx.items():
  idx_2_word[j] = i

In [ ]:
class EncoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
    super(EncoderLSTM, self).__init__()
    self.hidden_size = hidden_size

    self.num_layers = num_layers

    self.dropout = nn.Dropout(p)
    self.tag = True

    self.embedding = nn.Embedding(input_size, embedding_size)

    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)

  def forward(self, x):
    embedding = self.dropout(self.embedding(x))
    outputs, (hidden_state, cell_state) = self.LSTM(embedding)
    return hidden_state, cell_state


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_size_encoder = len(english.vocab)
encoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
encoder_dropout = 0.5

encoder_lstm = EncoderLSTM(input_size_encoder, encoder_embedding_size,
                           hidden_size, num_layers, encoder_dropout).to(device)
print(encoder_lstm)

EncoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(16201, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
)


In [ ]:
class DecoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p, output_size):
    super(DecoderLSTM, self).__init__()

    self.hidden_size = hidden_size
    self.num_layers = num_layers

    self.output_size = output_size

    # Regularization parameter
    self.dropout = nn.Dropout(p)

    self.embedding = nn.Embedding(input_size, embedding_size)
    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)

    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, x, hidden_state, cell_state):
    x = x.unsqueeze(0)

    embedding = self.dropout(self.embedding(x))
    outputs, (hidden_state, cell_state) = self.LSTM(embedding, (hidden_state, cell_state))
    predictions = self.fc(outputs)
    predictions = predictions.squeeze(0)
    return predictions, hidden_state, cell_state

In [ ]:
input_size_decoder = len(gujrati.vocab)
decoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
decoder_dropout = 0.5
output_size = len(gujrati.vocab)

decoder_lstm = DecoderLSTM(input_size_decoder, decoder_embedding_size,
                           hidden_size, num_layers, decoder_dropout, output_size).to(device)
print(decoder_lstm)

DecoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(22397, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  (fc): Linear(in_features=1024, out_features=22397, bias=True)
)


In [ ]:

for batch in train_iterator:
  print(batch.source.shape)
  print(batch.target.shape)
  break

x = batch.target[1]
print(x)

torch.Size([32, 32])
torch.Size([36, 32])
tensor([ 2374,     0,  1602,  1749,    16,   607,   126,    97,  1208, 13386,
           16,   218, 12013, 16762,     6,   138,  1058,   295,   644,  4869,
            0,     0,  1867,   287,    38,  4784, 11683,    14,  1623,  8822,
           34,  3613], device='cuda:0')


In [ ]:

class Seq2Seq(nn.Module):
  def __init__(self, Encoder_LSTM, Decoder_LSTM):
    super(Seq2Seq, self).__init__()
    self.Encoder_LSTM = Encoder_LSTM
    self.Decoder_LSTM = Decoder_LSTM

  def forward(self, source, target, tfr=0.5):
    batch_size = source.shape[1]

    target_len = target.shape[0]
    target_vocab_size = len(gujrati.vocab)

    outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)
    hidden_state, cell_state = self.Encoder_LSTM(source)

    x = target[0]

    for i in range(1, target_len):
      output, hidden_state, cell_state = self.Decoder_LSTM(x, hidden_state, cell_state)
      outputs[i] = output
      best_guess = output.argmax(1)
      x = target[i] if random.random() < tfr else best_guess
    return outputs

In [ ]:

# Hyperparameters

learning_rate = 0.001
step = 0

model = Seq2Seq(encoder_lstm, decoder_lstm).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

pad_idx = gujrati.vocab.stoi[""]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [ ]:
model

Seq2Seq(
  (Encoder_LSTM): EncoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(16201, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  )
  (Decoder_LSTM): DecoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(22397, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=1024, out_features=22397, bias=True)
  )
)

In [ ]:
def pred_trans(model, tokens, english, gujrati, device, max_length=100):
    tokens.insert(0, english.init_token)
    tokens.append(english.eos_token)
    text_to_indices = [english.vocab.stoi[token] for token in tokens]
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    with torch.no_grad():
        hidden, cell = model.Encoder_LSTM(sentence_tensor)

    outputs = [gujrati.vocab.stoi[""]]

    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.Decoder_LSTM(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)
        if output.argmax(1).item() == gujrati.vocab.stoi[""]:
            break

    translated_sentence = [gujrati.vocab.itos[idx] for idx in outputs]
    return translated_sentence[1:]

In [ ]:
import time
from tqdm import tqdm
epoch_loss = 0.0
num_epochs = 10
sentence1 = "avoid alcohol and illicit drugs"


for epoch in range(num_epochs):
  start_time = time.time()
  print("Epoch - {} / {}".format(epoch+1, num_epochs))
  model.eval()
  doc = nlp_en(sentence1)
  tok = [token.text for token in doc]
  translated_sentence1 = pred_trans(model, tok, english, gujrati, device, max_length=100)
  print(f"Translated example sentence 1: \n {translated_sentence1}")

  model.train(True)
  for batch_idx, batch in tqdm(enumerate(train_iterator), total=len(train_iterator)):
    input = batch.source.to(device)
    target = batch.target.to(device)

    output = model(input, target)
    output = output[1:].reshape(-1, output.shape[2])
    target = target[1:].reshape(-1)

    optimizer.zero_grad()

    loss = criterion(output, target)

    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

    optimizer.step()
    step += 1
    epoch_loss += loss.item()


  # Calculate the time taken for the epoch
  end_time = time.time()
  epoch_time = end_time - start_time
  print(f"Time taken for epoch {epoch + 1}: {epoch_time:.2f} seconds")
  print("Epoch_Loss - {}".format(loss.item()))
  print()

print(epoch_loss / len(train_iterator))



Epoch - 1 / 10
Translated example sentence 1: 
 ['દેશવ્યાપી', 'લોકોએ', 'ઓવર', 'સ્તૂપો', 'તકે', 'જનમાનસની', 'તને', 'સંભવિત', 'ટાગોરના', 'તને', 'રાખ્યાં', 'ઝાડના', 'રમતો', 'રમતો', 'કંડારવામાં', 'કંડારવામાં', 'કંડારવામાં', 'ટોચની', 'ટોચની', 'અભિષેક', 'વધવાને', 'વધવાને', 'વર્ષનું', 'વર્ષનું', 'વૈભવની', 'વૈભવની', 'વારસાનો', 'ખ્રિસ્તીઓની', 'સૂર્યપ્રકાશના', 'સૂર્યપ્રકાશના', 'વાતથી', 'વાતથી', 'આવરી', 'આવરી', 'કોલેસ્ટ્રોલના', 'હોસ્પિટલ', 'પરિસર', 'પરિસર', 'ગાળામાં', 'સૂર્યપ્રકાશના', 'ગાળામાં', 'સ્ટેડિયમ', 'સૂર્યપ્રકાશના', 'આગ્રા', 'રમતો', 'મંદિર', 'સ્તૂપો', 'દેલવાડા', 'આગ્રા', 'વહેતી', 'વહેતી', 'બેઠકો', 'રમતો', 'રમતો', 'રમતો', 'કંડારવામાં', 'કંડારવામાં', 'કંડારવામાં', 'ટોચની', 'ટોચની', 'અભિષેક', 'વધવાને', 'વધવાને', 'વર્ષનું', 'વર્ષનું', 'વૈભવની', 'વૈભવની', 'વારસાનો', 'ખ્રિસ્તીઓની', 'સૂર્યપ્રકાશના', 'સૂર્યપ્રકાશના', 'વાતથી', 'વાતથી', 'આવરી', 'આવરી', 'કોલેસ્ટ્રોલના', 'હોસ્પિટલ', 'પરિસર', 'પરિસર', 'ગાળામાં', 'સૂર્યપ્રકાશના', 'ગાળામાં', 'સ્ટેડિયમ', 'સૂર્યપ્રકાશના', 'આગ્રા', 'રમતો', 'મંદિર', 'સ્તૂપો

100%|██████████| 1464/1464 [06:57<00:00,  3.51it/s]


Time taken for epoch 1: 418.66 seconds
Epoch_Loss - 2.0833847522735596

Epoch - 2 / 10
Translated example sentence 1: 
 ['આ', 'અને', 'અને', 'અને', 'અને', 'છે', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<p

100%|██████████| 1464/1464 [06:56<00:00,  3.51it/s]


Time taken for epoch 2: 417.41 seconds
Epoch_Loss - 2.461972236633301

Epoch - 3 / 10
Translated example sentence 1: 
 ['અને', 'અને', 'અને', 'અને', 'છે', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', 

...

Time taken for epoch 8: 416.91 seconds
Epoch_Loss - 2.9727678298950195

Epoch - 9 / 10
Translated example sentence 1: 
 ['સેચ્યુરેટેડ', 'અને', 'અને', 'અને', 'અને', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>',

100%|██████████| 1464/1464 [06:57<00:00,  3.51it/s]


Time taken for epoch 9: 418.01 seconds
Epoch_Loss - 3.2310988903045654

Epoch - 10 / 10
Translated example sentence 1: 
 ['મેગ્નેશિયમ', 'અને', 'અને', 'આંતરડાને', 'બનાવે', 'છે', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pa

100%|██████████| 1464/1464 [06:55<00:00,  3.53it/s]

Time taken for epoch 10: 415.67 seconds
Epoch_Loss - 3.1189868450164795

33.02071694215464


In [ ]:
s = test_source[10]
translation = translate_sentence(model, s, english, gujrati, device, max_length=100)
cleaned_tokens = [token for token in translation if token not in ['<eos>', '<pad>']]
readable_translation = ' '.join(cleaned_tokens)
print(readable_translation)

ગુફા ગુફા પંચવટી એક ગુફા છે


In [ ]:
gujrati_pred = []

for tokens in tokenized_test_source:
  translation = pred_trans(model, tokens, english, gujrati, device, max_length=100)
  cleaned_tokens = [token for token in translation if token not in ['<eos>', '<pad>','।','৷']]
  pred = ' '.join(cleaned_tokens)
  print(pred)
  gujrati_pred.append(pred)


Streaming output truncated to the last 5000 lines.
ભારતીય અને અને અને અને અને અને અને અને અને અને અને અને અને અને અને અને અને અને અને અને
શું આ આ આવડે
આ નાયિકાઓ અથવા અથવા લોઅર અથવા આ અથવા પણ પણ છે
હવે આપણે અમે આપણે જ જાણકારીઓ આપીએ છીએ
પોતાના શ્વાસ અને અને અને અને જ જ છે
આ આ મહિને એક સહેલાઈથી કરી શકાય છે
સૂરજ કોન્ટેક્ટને નેવું રમો
જ્યારે એક એક જ છે છે છે જ્યારે તે હોય છે
આ જ આ છે કે આ સ્થાનને અહીં અયોધ્યામાં અયોધ્યામાં અયોધ્યામાં પણ છે
મારે નંબર રૂપિયા એકાઉન્ટ રૂપિયા મારો શોધવાનો પ્રયાસ કરી શકો શું આપ મારું કોવીડ કોવીડ સર્ટિફિકેટ ડાઉનલોડ કરી શકો
આ આનુવંશિકતા રીતે મગફળીને છે છે છે અને અને અને અને છે અને છે
બાળકો માટે માટે માટે માટે જ ધ્યાન રાખવું જોઈએ
હવે એક એક એક રીતે એક એક હતું કે આ આ પછી આ પર પર
પં જન્મ ૩૦ જાન્યુઆરી રામચંદ્ર રામચંદ્ર વારાણસીમાં રોજ રોજ થયો
સ્વ તમિલ બૉલીવુડના ગ્વાલિયરને કોલકાતામાં છે જે ડેલી વંશના સ્વરૂપ છે છે
આ આ આ આ દવાને આ આ આઈસ અનુપમ છે
સાથે જ જ અને ચોકલેટ જાનવરોની અને જેમકે પણ પણ અને અને અને અને અને અને વગેરે અને અને અને પેરા વગેરે અને અને અને અને વગેરે પણ પણ શકે 

In [ ]:
import csv

# Create a list of dictionaries where each dictionary represents a row
data = [{'valid_id': valid_id[i], 'gujrati_pred': gujrati_pred[i]} for i in range(len(valid_id))]

# Specify the CSV file path
csv_file_path = '/gdrive/MyDrive/gujrati3.csv'

# Define the column names
fields = ['valid_id', 'gujrati_pred']

# Write the data to the CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fields)
    writer.writeheader()
    writer.writerows(data)

print(f'Saved predictions to {csv_file_path}')

NameError: ignored

In [ ]:
print(len(gujrati_pred))
print(len(test_source))

13567
13567
